In [13]:
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.linear_model import Ridge

from sklearn.impute import SimpleImputer

In [15]:
df = pd.read_csv('https://raw.githubusercontent.com/jbyrneirl/BER-Rating-AI-Model/main/data/training/BERRatingData_aa.csv', sep=';', on_bad_lines="skip", low_memory=False)

np.random.seed(0)

X = df.drop("BerRating", axis='columns')
X = pd.get_dummies(X)

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(X)
X = imp.transform(X)
y = df.BerRating

#add feature reduction

X_train, X_test, y_train, y_test = train_test_split( X, y, random_state=2)

c:\Users\Aidan\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\impute\_base.py:577: UserWarning: Skipping features without any observed values: ['SecondEnerConsumedComment' 'ThirdEnerConsumedComment']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


In [16]:
model = Ridge()


model.fit(X_train , y_train)
score = model.score(X_test , y_test)

In [17]:
print(score)

0.9947700444335452
